In [ ]:
import pandas as pd
import numpy as np

from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import quandl
quandl.ApiConfig.api_key = "q2DtgvtfT_ZV2DbsFyCF"

In [ ]:
## La siguiente función puede utilizarse para analizar tres gráficos: la serie, la función 
## autocorrelación y la función de autocorrelación parcial

def tsplot(y, lags=None, figsize=(15, 10), style='bmh', titulo = 'Time Series Analysis Plots'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        #qq_ax = plt.subplot2grid(layout, (2, 0))
        #pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(titulo)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.01)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.01)
        #sm.qqplot(y, line='s', ax=qq_ax)
        #qq_ax.set_title('QQ Plot')        
        #scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
# Un posible sitio para descargar series financieras es quandl.com, actualmente trasladado a nasdaq.com
#El comando para descarga es quandl.get(). Algunas series son de libre descarga (Free), y otras 
#requieren un pago. (Premium)

data = quandl.get("NSE/SPYL", start_date="2011-01-12", end_date="2013-01-01")
len(data)

Otro data sets para series se puede obtener del sitio https://faculty.chicagobooth.edu/ruey-s-tsay/teaching

Estas series fueron utilizadas en las diferentes ediciones del libro Analysis of Financial Time Series, de Ruey S. Tsay.

- Monthly IBM stock, VW index, EW index and S&P index returns: m-ibm3dx2608.csv

- Growth rate of U.S. quarterly real gnp: dgnp82.txt

- Daily values of S&P 500 index: d-sp55008.csv

- Monthly returns of VW index: VW_index_rets.csv

- Weekly 1-yr & 3-yr interest rates: w-gs1yr.csv & w-gs3yr.csv

- Monthly simple returns of Intel stock: Intel_returns.csv

- 10-minute FX log returns (Mark-Dollar): FX_logret_Mark_Dollar.csv

- Monthly excess returns of the S&P 500 index: S_P500_index.csv

In [ ]:
rates_1yr = pd.read_csv("Data/w-gs1yr.csv")
rates_3yr = pd.read_csv("Data/w-gs3yr.csv")

IBM_VW_EW_SyP_index = pd.read_csv("Data/m-ibm3dx2608.csv")
IBM_VW_EW_SyP_index['date'] = pd.to_datetime(IBM_VW_EW_SyP_index['date'].astype(str), format ='%Y%m%d' )

GNP = pd.read_csv("Data/dgnp82.csv")
S_P_index_d = pd.read_csv("Data/d-sp55008.csv")
S_P500_index = pd.read_csv("Data/S_P500_index.csv")

VW_Index_rets = pd.read_csv("Data/VW_Index_rets.csv")

FX_logret_mark_dollar = pd.read_csv("Data/FX_logret_Mark_Dollar.csv")

Intel_returns = pd.read_csv("Data/Intel_returns.csv")
Intel_returns['date'] = pd.to_datetime(Intel_returns['date'].astype(str), format='%Y%m%d')

### Modelos autorregresivos de orden p $AR(p)$

Es un modelo de regresión que depende linealmente de términos anteriores:
$$x_t = \phi_1x_{t-1}+…+\phi_px_{t-p}+a_t = \sum_{i=1}^{p} t_i\phi_ix_{t-i}+a_t$$

El orden "p" representa el número de términos anteriores (retardos) usados dentro del modelo. $\phi_i$ es el coeficiente del retardo $i$ y $a_t$ es un ruido blanco.

Un modelo AR(1) con $\phi_1=1$ es una caminata aleatoria.

Modelamos un AR(1) $\phi$ igual a 0.6

In [ ]:

# Simulamos un AR(1) con phi_1 = 0.6
## Modificamos ph1_1=1, phi_1=1.01, phi_1=-0.3
n = int(1000)
phis = np.array([0.6])
thetas = np.array([0.])

# Python requiere especificar el valor de lag 0: 1
# Notar que se ingresa -phi: coeficientes del polinomio autorregresivo.
# Los coeficientes theta (del modelo MA) son 0 para un AR
ar = np.r_[1, -phis]
ma = np.r_[1, thetas]

ar1 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(ar1, lags=30, titulo = "AR(1)")

Ajustamos la serie anterior a un AR(1). Observamos algunos resultados arrojados:

- Tabla 1: Criterios de información, métodos de ajuste.
- Tabla 2: Parámetros estimados. Media y desvío estándar.
- Tabla 3: Raíces características.

In [ ]:
#Ajustamos la serie al modelo AR(1), utilizando AutoReg.
mod = AutoReg(ar1, 1, old_names=False)
res = mod.fit()

print(res.summary())


In [ ]:
## ajuste con SARIMAX
mdl = sm.tsa.arima.ARIMA(x, order = (1,0,0))
res = mdl.fit()
print(res.summary())

In [ ]:
##seleccionamos el orden del model AR.

sel = ar_select_order(ar1, 8, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:
## Predicción: predice el último valor tomado.
res.plot_predict(start=951, end=1010)
plt.plot(ar1[950:1000])
plt.show()

### SIMULACIÓN DE UN PROCESO AR(2)

$$ x_t = \phi_1 x_{t-1} + \phi_2 x_{t-2} + a_t$$

In [ ]:
# Simulamos un AR(2) 

n = int(1000)
phis = np.array([0.666, -0.333])
thetas = np.array([0.])

# Python requiere especificar el valor de lag 0: 1
# Notar que se ingresa -phi: coeficientes del polinomio autorregresivo.
# Los coeficientes theta (del modelo MA) son 0 para un AR
ar = np.r_[1, -phis]
ma = np.r_[1, thetas]

ar2 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(ar2, lags=30, titulo = "AR(2)")

In [ ]:
mod = AutoReg(ar2, 2, old_names=False)
res = mod.fit()

print(res.summary())

Hay una correlación significativa en los dos primeros retardos. Ver el PACF. Ajustamos los datos

In [ ]:
mdl = sm.tsa.arima.ARIMA(ar2, order = (3,0,0))
res = mdl.fit()
print(res.summary())

In [ ]:
sel = ar_select_order(x, 8, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:
#data = VW_Index_rets["Retornos"]
#_ = tsplot(data, lags=20, titulo = "VW INDEX")

data = S_P500_index["Monthly excess returns"]
_ = tsplot(data, lags=20, titulo = "S&P INDEX")

In [ ]:
sel = ar_select_order(data, 8, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:
mdl = sm.tsa.arima.ARIMA(data, order = (3,0,0))
res = mdl.fit()
print(res.summary())

In [ ]:
sel = ar_select_order(data, 5, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:

res.predict(start = 751, end = 800).plot()
plt.plot(data[750:])
plt.show()

In [ ]:
# Simulación de un ARIMA(2,1,1) con phis=[0.5,-0.25] and thetas=[-0.5]

max_lag = 30

n = int(2000)
burn = 100

phis = np.array([0.5,-0.25])
thetas = np.array([-0.5])

ar = np.r_[1, -phis]
ma = np.r_[1, thetas]

arma11 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n, burnin=burn)
arima111 = arma11.cumsum()

_ = tsplot(arima111, lags=max_lag, titulo = "ARIMA(1,1,1)")
#_ = tsplot(arma11, lags=max_lag, titulo = "ARMA(1,1)")


In [ ]:
sel = ar_select_order(arma11, 8, glob=True, old_names=False)
sel.ar_lags
res = sel.model.fit()
print(res.summary())

In [ ]:
# Ajuste a  ARIMA(p, d, q) 
# elección del mejor orden basado en aic

best_aic = np.inf 
best_order = None
best_mdl = None

pq_rng = range(5) # [0,1,2,3,4]
d_rng = range(2) # [0,1]
for i in pq_rng:
    for d in d_rng:
        for j in pq_rng:
            try:
                tmp_mdl = sm.tsa.arima.ARIMA(arima111, order=(i,d,j)).fit()
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue


print('aic: %6.5f | order: %s'%(best_aic, best_order))

# ARIMA model resid plot
_ = tsplot(best_mdl.resid, lags=30, titulo = "residuos")

In [ ]:
tmp_mdl = sm.tsa.arima.ARIMA(arima111, order=(1,1,1)).fit()
print(tmp_mdl.summary())

In [ ]:
_ = tsplot(tmp_mdl.resid, lags=30, titulo = "residuos")